Credits: https://www.kaggle.com/carlolepelaars/bidirectional-lstm-for-audio-labeling-with-keras

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Bidirectional, CuDNNLSTM
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import ELU
from keras.utils import np_utils
from keras import optimizers, metrics

from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers import Embedding, Input, Dense, CuDNNGRU,concatenate, Bidirectional, SpatialDropout1D, Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
train_dataX = np.load('augmentedData_Combined.npy')
test_dataX = np.load('test_data.npy')
train_datay = pd.read_csv('AugmentedCombined_labels.csv')

#pre-processing data
train_datay = train_datay.drop('Id',1)
train_dataX = train_dataX.reshape(-1,210,210)
test_dataX = test_dataX.reshape(-1,210,210)
Y_train = np_utils.to_categorical(train_datay,2)

#splitting data for validation
X_train, X_test, y_train, y_test = train_test_split(train_dataX,Y_train, test_size=0.1, shuffle=True)

In [3]:
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [5]:
#del model
model = Sequential()
model.add(Bidirectional(CuDNNLSTM(256, return_sequences=True), input_shape=(210,210)))
model.add(Attention(210))
model.add(Dropout(0.2))
model.add(Dense(400))
model.add(ELU())
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 210, 512)          958464    
_________________________________________________________________
attention_1 (Attention)      (None, 512)               722       
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 400)               205200    
_________________________________________________________________
elu_1 (ELU)                  (None, 400)               0         
_________________________________________________________________
dropout_2 (Dropout)  

In [6]:
adam = optimizers.Adam(lr=0.01)
model.compile(optimizer=adam,
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [7]:
model.fit(X_train, y_train,
          batch_size=250, nb_epoch=80,verbose=1,
          validation_data=(X_test, y_test))

Instructions for updating:
Use tf.cast instead.


d:\programs\python installed\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Train on 21594 samples, validate on 2400 samples
Epoch 1/80
21594/21594 [==============================] - 32s 1ms/step - loss: 1.9331 - acc: 0.8731 - val_loss: 2.0551 - val_acc: 0.8725
Epoch 2/80
21594/21594 [==============================] - 20s 919us/step - loss: 1.9571 - acc: 0.8786 - val_loss: 2.0551 - val_acc: 0.8725
Epoch 3/80
 9750/21594 [============>.................] - ETA: 9s - loss: 1.9457 - acc: 0.8793

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import roc_auc_score

Y_test_predClass = model.predict_proba(X_test)

roc = roc_auc_score(y_test, Y_test_predClass)
print("ROC:" + str(round(roc,4)))

In [ ]:
Y_test_pred = model.predict_proba(test_dataX)
print(Y_test_pred)

In [ ]:
pd.DataFrame(Y_test_pred[:,0]).to_csv('submission55.csv', header=["Label"], index_label="Id")